<a href="https://colab.research.google.com/github/DiegoRenner/Game-Sim-ML-in-Finances-/blob/main/full_tf_model/training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf Game-Sim-ML-in-Finances-
!rm -rf log
!git clone https://github.com/DiegoRenner/Game-Sim-ML-in-Finances-

Cloning into 'Game-Sim-ML-in-Finances-'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 209 (delta 113), reused 144 (delta 63), pack-reused 0
Receiving objects: 100% (209/209), 3.29 MiB | 6.35 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [2]:
import sys
sys.path.insert(0,'/content/Game-Sim-ML-in-Finances-/full_tf_model')

In [3]:
from env_total_v2 import Game
from utils import create_input_batch
from train import train


# Hyperparameters
game_params = {
    "end_time": 11,  # number of steps in each game
    "agent_num": 2,
}

training_params = {
    "epochs_total": 4,  # number of times each agent is trained
    "population_size": 10,  # number of different weights compared
    "max_iterations": 20,  # number of iterations optimization algorithm runs
    "log_path": "log/game/",
    "save": False,
}

batch_params = {
    "batch_size": 4,  # number of epochs an agent is trained per training
    "irm_pool": [0.01, 0.02, 0.05],
    "irs_pool": [0.0],
    "divm_pool": [0.5, 1, 1.5],
    "divs_pool": [0.2, 0.3, 0.4],
    "endow_cash_pool": [20, 100, 80],
    "endow_stock_pool": [10, 15, 10],
}

log_params = {**game_params, **training_params, "batch_size":
              batch_params["batch_size"]}

game = Game(**game_params)
batch = create_input_batch(game.agent_num, game.end_time, **batch_params)

# Dummy pass to ensure initialization
x = batch[0]
_ = game(x)

print(game.train)

train(game, batch, **training_params, log_params=log_params)

  0%|          | 0/10 [00:00<?, ?population_element/s]

True
---------------------------------------
Training output:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
EPOCHS_TOTAL: 0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####################### Training agent: 0 ######################
Training iteration 0


  0%|          | 0/10 [00:00<?, ?population_element/s]

Training iteration 1


  0%|          | 0/10 [00:00<?, ?population_element/s]

...

  0%|          | 0/10 [00:00<?, ?population_element/s]

Training iteration 19


  0%|          | 0/10 [00:00<?, ?population_element/s]

Training iteration 20


100%|██████████| 10/10 [00:06<00:00,  1.48population_element/s]


In [4]:

!tensorboard dev upload --logdir ./log/game/ \
  --name "MLF - Game Simulation" \
  --description "Results from https://github.com/DiegoRenner/Game-Sim-ML-in-Finances-/blob/main/full_tf_model/training_notebook.ipynb" \
  --one_shot

2021-05-30 22:40:53.049536: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/2uWTRko4R4ulGTxzKkuwOA/

[2021-05-30T22:40:54] Started scanning logdir.
[2021-05-30T22:40:54] Total uploaded: 32 scalars, 1 tensors (6 B), 0 binary objects
[2021-05-30T22:40:54] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/2uWTRko4R4ulGTxzKkuwOA/
